# Using ZhipuAI GLM-4 though OpenAI SDK and Ecosystem

**This tutorial is available in English and is attached below the Chinese explanation**

本节代码将带领大家使用`OpenAI`的 SDK 和支持的框架调用 `GLM-4` 模型
你将在本章节学习到如何使用 Langchain 的 `ChatOpenAI` 类来调用 ZhipuAI 的 GLM-4 模型，以及如何使用 `OpenAI SDK` 类来实现调用GLM-4模型。

This notebook section will walk you through calling `GLM-4` models using the `OpenAI` SDK and supporting frameworks.
In this section you will learn how to call ZhipuAI's GLM-4 models using Langchain's `ChatOpenAI` class, and how to use the `OpenAI SDK` class to implement calling GLM-4 models.


## 1. Use OpenAI SDK

首先，需要导入 `OpenAI` SDK，同时，修改 `OpenAI` 的 `base_url` 为 `https://open.bigmodel.cn/api/paas/v4/` 
以便调用 `GLM-4` 模型。完成更换后，仅需填写 Key 就能使用 `GLM-4` 模型。

First, you need to import the `OpenAI` SDK and change the `OpenAI` `base_url` to `https://open.bigmodel.cn/api/paas/v4/` in order to call the `GLM-4` model. 
in order to call the `GLM-4` model. Once you have done this, you can use the `GLM-4` model by simply filling in the Key.
``python

In [5]:
!pip install openai langchain_openai -i https://pypi.org/simple

   ---------------------------------------- 0.0/798.7 kB ? eta -:--:--
   - -------------------------------------- 30.7/798.7 kB 1.3 MB/s eta 0:00:01
   - -------------------------------------- 30.7/798.7 kB 1.3 MB/s eta 0:00:01
   ----- -------------------------------- 122.9/798.7 kB 901.1 kB/s eta 0:00:01
   ------- ------------------------------ 153.6/798.7 kB 833.5 kB/s eta 0:00:01
   ------- ------------------------------ 153.6/798.7 kB 833.5 kB/s eta 0:00:01
   ------- ------------------------------ 153.6/798.7 kB 833.5 kB/s eta 0:00:01
   --------- ---------------------------- 194.6/798.7 kB 562.0 kB/s eta 0:00:02
   ------------ ------------------------- 256.0/798.7 kB 684.6 kB/s eta 0:00:01
   ------------- ------------------------ 276.5/798.7 kB 710.0 kB/s eta 0:00:01
   ------------- ------------------------ 286.7/798.7 kB 655.2 kB/s eta 0:00:01
   ------------- ------------------------ 286.7/798.7 kB 655.2 kB/s eta 0:00:01
   -------------- ----------------------- 307.2/798

In [2]:
from openai import OpenAI
client = OpenAI(
    api_key="7cc2d454b5b31371f5c6c990fddc7636.EPXYYOmeKYWvDEqW",
    base_url="https://open.bigmodel.cn/api/paas/v4/")

## 2. Simple use of OpenAI SDK to call GLM-4

我将带领大家完成最基础的对话调用。
请注意：
- `temperature` 参数的区间为 (0,1)。
- `do_sample = False (temperature = 0)` 在 OpenAI 调用中并不适用。
-  异步操作不适用。

其他例子(例如工具调用)，也均可以按照OpenAI的方式进行调用。


I will lead everyone to complete the most basic dialogue calls.
Please note:

- The interval of the `temperature` parameter is (0,1).
- `do_sample=False (temperature=0)` is not applicable in OpenAI calls.
- Asynchronous operations are not applicable.

Other examples (such as tool calls) can also be called in the same way as OpenAI.

In [3]:
response = client.chat.completions.create(
    model="glm-4",
    messages=[
        {
            "role": "user",
            "content": "tell me a joke"
        }
    ],
    top_p=0.7,
    temperature=0.9,
    stream=False,
    max_tokens=2000,
)
response

ChatCompletion(id='8605592118730949543', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure! Here's a light-hearted joke for you:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!", role='assistant', function_call=None, tool_calls=None))], created=1714214949, model='glm-4', object=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=9, total_tokens=35), request_id='8605592118730949543')

## 3. Using Langchain's ChatOpenAI class

Langchain 的 `ChatOpenAI` 类是对 `OpenAI` SDK 的封装，可以更方便调用。这里展示了如何使用 `ChatOpenAI` 类来调用 `GLM-4` 模型。

Langchain's `ChatOpenAI` class is a wrapper around the `OpenAI` SDK, making it easier to call. Here we show how to use the `ChatOpenAI` class to call the `GLM-4` model.

In [7]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature=0.95,
    model="glm-4",
    openai_api_key="your zhipuai api key",
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)
conversation.invoke({"question": "tell me a joke"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: tell me a joke


AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': '令牌已过期或验证不正确！'}}

## 4. Calling GLM-4 with LangChain AgentExecutor
同时，GLM-4也能很好的接入到 LangChain 的 AgentExecutor 中，这里展示了如何使用 `AgentExecutor` 来调用 `GLM-4` 模型。

GLM-4 also plugs nicely into LangChain's AgentExecutor, which shows how to call a `GLM-4` model using `AgentExecutor`.

In [8]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["TAVILY_API_KEY"] = "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"
tools = [TavilySearchResults(max_results=2)]
prompt = hub.pull("hwchase17/react")
# Choose the LLM to use
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': '令牌已过期或验证不正确！'}}